In [ ]:
import pandas as pd
import os
import json
import numpy as np
from itertools import groupby
import matplotlib.pyplot as plt
from scipy import stats,signal
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
import random
import re
import csv
from IPython.display import HTML, display, Image
import tabulate
import math as m
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.formula.api as smf
import itertools

mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.linewidth'] = 1
#mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 1
#mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.width'] = 1
mpl.rcParams['ytick.minor.width'] = 1
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

fday = [1,2,3,4,5,6,7,8,9,10]
fsession = [1,2]
unique_signals = [1,2,3,4]

In [ ]:
current_path = os.path.abspath(os.getcwd())
parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
grand_parent_path = os.path.abspath(os.path.join(parent_path, os.pardir))
main_path = os.path.abspath(os.path.join(grand_parent_path, os.pardir))

path_results = main_path+'/results/dots/'

In [ ]:
adf = pd.read_csv(path_results+'preanalyzed.csv')  
df_diff = pd.read_csv(path_results+'per_difficulty.csv')  
dfANOVA = pd.read_csv(path_results+'forANOVA.csv') 
dfLR = pd.read_csv(path_results+'linearRegr_diff.csv') 

adf = pd.merge(dfLR,adf,on='user_sessionID',how='outer') 
# sort df by userID and sessionID in second place
adf = adf.sort_values(by = ['userID', 'sessionID_y'])

userids = adf['userID'].unique()
userids = sorted(userids)
nsub = len(userids)
sessionids = adf['sessionID_x'].unique()
sessionids = sorted(sessionids)

# morning df
mdf = adf[adf['sessionID_x']%2==1]
# evening df
edf = adf[adf['sessionID_x']%2==0]
'''
edf_sin_nan = edf.copy()
edf_sin_nan['sleep']=list(mdf['sleep'])
'''

key_SR = ['mood','food','sleep', 'real_stress']
key_PV = ['Dsubj_optout_oo','Ssubj_optout_oo','subj_perf_no','OKubj_RT_no','DoverConf','SoverConf','risk_av']
key_LR = ['slope_RT', 'slope_DO', 'slope_SO', 'intercept_RT', 'intercept_DO','intercept_SO']
key_name = key_SR+key_PV

key_SR2plot = [key_SR[:int(len(key_SR)/2)],key_SR[int(len(key_SR)/2):]]

SR_color = {'mood': 'b','food': 'r','sleep': 'y', 'real_stress':'g'}

pair_keys_SR = list(itertools.combinations(key_SR, 2))

# pair of PV 
pair_keys_PV = list(itertools.combinations(key_PV[:-3], 2))
pair_keys_PV2plot = [pair_keys_PV[:int(len(pair_keys_PV)/2)],pair_keys_PV[int(len(pair_keys_PV)/2):]]
key_name_PV = {'Dsubj_optout_oo':'DO','Ssubj_optout_oo':'SO','subj_perf_no':'acc NO','OKubj_RT_no':'RT NO'}
pair_key_name_PV2plot = [[[key_name_PV[k] for k in tupl] for tupl in lista] for lista in pair_keys_PV2plot]

# avoiding participants who answered almost the same SR across sessions
users_SR = {'food':[x for x in userids if x!=1010 and x!=1014],'real_stress':[x for x in userids if x!=997],\
            'mood':userids,'sleep':[x for x in userids if x!=1005]}

key_diff = ['Dperf_oo', 'Sperf_oo', 'perf_no', 'Dn_trials_oo', 'Sn_trials_oo', 'NOn_trials', 'DRT_oo', 'SRT_oo', \
            'DRT_OKoo', 'SRT_OKoo', 'RT_no', 'RT_noNOK', 'Doptout', 'Soptout']

In [ ]:
adf['day_time'] = adf.sessionID_x.map(lambda x: 'afternoon' if x%2==0 else 'morning')

In [ ]:
adf.head()

In [ ]:
mVSa_test = {}
for part in userids:
    subset = adf[adf['userID']==part]
    dict_ = {}
    for key in key_SR+key_PV:
        if key != 'sleep':
            dict_[key] = stats.ttest_rel(subset[subset['day_time']=='morning'][key],\
                                             subset[subset['day_time']=='afternoon'][key])
    mVSa_test[part]=dict_

In [ ]:
df = pd.DataFrame(mVSa_test)

In [ ]:
totalRTno_p = []
for part in userids:
    subset = adf[adf['userID']==part]
    lala = stats.ttest_rel(subset[subset['day_time']=='morning']['subj_RT_no'],\
                                             subset[subset['day_time']=='afternoon']['subj_RT_no'])
    totalRTno_p.append(lala.pvalue)

In [ ]:
plt.hist(totalRTno_p,bins=20)
plt.axvline(0.05,color='k')
plt.xlabel('p value')
plt.ylabel('total RT no counts')
plt.show()

In [ ]:
RTno_morning = adf[adf['day_time']=='morning']['subj_RT_no']
RTno_afternoon = adf[adf['day_time']=='afternoon']['subj_RT_no']

In [ ]:
adf['subj_RT_no'].isna().sum()

In [ ]:
RTno_morning = RTno_morning[~np.isnan(RTno_morning)]

In [ ]:
np.isnan(RTno_morning).any()

In [ ]:
len(RTno_morning)

In [ ]:
RTno_afternoon = RTno_afternoon[~np.isnan(RTno_afternoon)]

In [ ]:
len(RTno_morning)

In [ ]:
len(RTno_afternoon)

In [ ]:
RTno_afternoon = list(RTno_afternoon)+[np.mean(RTno_afternoon)]

In [ ]:
len(RTno_afternoon)

In [ ]:
stats.ttest_rel(RTno_morning,RTno_afternoon)

In [ ]:
RTnoOK_morning = adf[adf['day_time']=='morning']['OKubj_RT_no']
RTnoOK_afternoon = adf[adf['day_time']=='afternoon']['OKubj_RT_no']

In [ ]:
RTnoOK_morning = RTnoOK_morning[~np.isnan(RTnoOK_morning)]

In [ ]:
len(RTnoOK_morning)

In [ ]:
RTnoOK_afternoon = RTnoOK_afternoon[~np.isnan(RTnoOK_afternoon)]

In [ ]:
len(RTnoOK_afternoon)

In [ ]:
RTnoOK_afternoon = list(RTnoOK_afternoon)+[np.mean(RTnoOK_afternoon)]

In [ ]:
stats.ttest_rel(RTnoOK_morning,RTnoOK_afternoon)

In [ ]:
plt.hist(RTno_afternoon,alpha=0.5)
plt.hist(RTno_morning,alpha=0.5)
plt.title(stats.ttest_rel(RTno_morning,RTno_afternoon))
plt.legend(('afternoon','morning'))
plt.xlabel('total RT non-optout')
plt.tight_layout()
plt.savefig('distr_totalRTno_mVSa.png')

plt.show()

In [ ]:
RTnoNOK_morning = adf[adf['day_time']=='morning']['NOKubj_RT_no']
RTnoNOK_afternoon = adf[adf['day_time']=='afternoon']['NOKubj_RT_no']

In [ ]:
RTnoNOK_morning = RTnoNOK_morning[~np.isnan(RTnoNOK_morning)]

In [ ]:
RTnoNOK_afternoon = RTnoNOK_afternoon[~np.isnan(RTnoNOK_afternoon)]

In [ ]:
RTnoNOK_afternoon = list(RTnoNOK_afternoon)+[np.mean(RTnoNOK_afternoon)]

In [ ]:
stats.ttest_rel(RTnoNOK_morning,RTnoNOK_afternoon)

In [ ]:
plt.hist(RTnoOK_afternoon,alpha=0.5)
plt.hist(RTnoOK_morning,alpha=0.5)
plt.title(stats.ttest_rel(RTnoOK_morning,RTnoOK_afternoon))
plt.legend(('afternoon','morning'))
plt.xlabel('correct RT non-optout')
plt.tight_layout()
plt.savefig('distr_correctRTno_mVSa.png')

plt.show()

In [ ]:
plt.hist(RTnoNOK_afternoon,alpha=0.5)
plt.hist(RTnoNOK_morning,alpha=0.5)
plt.title(stats.ttest_rel(RTnoNOK_morning,RTnoNOK_afternoon))
plt.legend(('afternoon','morning'))
plt.xlabel('incorrect RT non-optout')
plt.tight_layout()
plt.savefig('distr_incorrectRTno_mVSa.png')

plt.show()

In [ ]:
adf[adf['NOKubj_RT_no']>2][['userID','real_OK_NO_RT','real_OK_DO_RT',\
                            'real_OK_SO_RT','real_NOK_NO_RT','real_DO_RT','real_SO_RT']]

In [ ]:
perf_no_morning = adf[adf['day_time']=='morning']['subj_perf_no']
perf_no_afternoon = adf[adf['day_time']=='afternoon']['subj_perf_no']

In [ ]:
perf_no_morning = perf_no_morning[~np.isnan(perf_no_morning)]

In [ ]:
perf_no_afternoon = perf_no_afternoon[~np.isnan(perf_no_afternoon)]

In [ ]:
perf_no_afternoon = list(perf_no_afternoon)+[np.mean(perf_no_afternoon)]

In [ ]:
stats.ttest_rel(perf_no_morning,perf_no_afternoon)